In [1]:
import random
import numpy as np
import svgwrite

In [2]:
def adj(nodea, nodeb):
    return sum([abs(nodea[i] - nodeb[i]) for i in range(len(nodea))]) == 1

In [32]:
# 101x101 = 7845 nodes
# 72x72x2 = 7880 nodes 
N = 69 # rows
M = 69 # columns

# nodes = [(i,j,k) for i in range(N) for j in range(M) for k in [0,1]]
nodes = [(i,j) for i in range(N) for j in range(M)]
# edges = list()
# for i in range(N - 1):
#     for j in range(M - 1):
#         edges.append(((i,j),(i+1,j)))
#         edges.append(((i,j),(i,j+1)))

# for i in range(N - 1):
#     edges.append(((i, M-1), (i+1, M-1)))

# for j in range(M - 1):
#     edges.append(((N-1, j), (N-1, j+1)))
    
nodes = list(filter(lambda x: (x[0]-((N-1)/2))**2 + (x[1]-((N-1)/2))**2 <= ((N-1)/2)**2, nodes))
edges = [(i, j) for i in nodes for j in nodes if adj(i,j) and (i[0] <= j[0] and i[1] <= j[1])]
len(nodes), len(edges)

(3625, 7112)

In [ ]:
no = len(nodes)
laplacian = np.zeros((no,no), dtype = np.int8)
for edge in edges:
    idx1 = nodes.index(edge[0])
    idx2 = nodes.index(edge[1])
    laplacian[idx1][idx1] += 1
    laplacian[idx2][idx2] += 1
    laplacian[idx1][idx2] -= 1
    laplacian[idx2][idx1] -= 1

In [ ]:
laplacian
eigs = np.linalg.eig(laplacian)
print(np.prod(eigs[0])/(eigs[0][1] * len(nodes)))
eigs[0]

In [33]:
def find_and_update_parent(parents, idx):
    if (parents[idx] == idx):
        return idx
    else:
        parents[idx] = find_and_update_parent(parents, parents[idx])
        return parents[idx];

def merge(parents, idx1, idx2):
    parent1 = find_and_update_parent(parents, idx1)
    parent2 = find_and_update_parent(parents, idx2)
    parents[max(parent1, parent2)] = min(parent1, parent2)
    

def connected(parents, idx1, idx2):
    return find_and_update_parent(parents, idx1) == find_and_update_parent(parents, idx2)

def random_tree(nodes, edges, seed=[]):
    parents = list(range(len(nodes)))
    shuffled = random.sample(edges, len(edges))
    look_through = seed + shuffled
#     print(shuffled)
    tree = []
    for edge in look_through:
        node1 = edge[0]
        node2 = edge[1]
        idx1 = nodes.index(node1)
        idx2 = nodes.index(node2)
        if not connected(parents, idx1, idx2):
#             print(node1, node2, idx1, idx2)
            tree.append(edge)
            merge(parents, idx1, idx2)
#             print(parents)
    assert len(tree) == len(nodes) - 1
    return tree

def draw(tree, N, M, seed=[]):
    wall_char = "█"
    arr = [[wall_char for i in range(2*M - 1)] for j in range (2*N - 1)]
    nodes = set([edge[0] for edge in tree] + [edge[1] for edge in tree])
    for node in nodes:
        arr[node[0] * 2][node[1] * 2] = ' '
#     for i in range(0, 2*N - 1, 2):
#         for j in range(0, 2*M - 1, 2):
#             arr[i][j] = ' '
    for edge in tree:
        node1 = edge[0]
        node2 = edge[1]
        if node1[0] == node2[0]:
            # horizontal
            i = 2*node1[0]
            j = node1[1] + node2[1]
            arr[i][j] = " "
        else:
            # vertical
            i = node1[0] + node2[0]
            j = 2*node1[1]
            arr[i][j] = " "
    
    for edge in seed:
        node1 = edge[0]
        node2 = edge[1]
        if node1[0] == node2[0]:
            # horizontal
            i = 2*node1[0]
            j = node1[1] + node2[1]
            arr[i][j] = "|"
        else:
            # vertical
            i = node1[0] + node2[0]
            j = 2*node1[1]
            arr[i][j] = "-"
    
    binrep = []
    for i,arri in enumerate(arr):
        if i%2 == 0:
            binrep.append(int("".join(["1" if arri[2*j + 1] == ' ' else "0" for j in range(M-1)]), 2))
        else:
            binrep.append(int("".join(["1" if arri[2*j] == ' ' else "0" for j in range(M)]), 2))
                
    stmp = (wall_char * (2*M + 1) + "\n") + "\n".join([wall_char + "".join(arri) + wall_char for arri in arr]) + ("\n" + wall_char * (2*M + 1))
    sarr = list(stmp)
#     sarr[2*M + 2] = " "
#     sarr[-1*(2*M + 3)] = " "
    
    return "".join(sarr), binrep, 

def drawsvg(tree, fname):
    dwg = svgwrite.Drawing(fname, profile='full')
    for edge in tree:
        mm = svgwrite.mm # 0.135 in
        start = (3.429 * edge[0][0] * mm, 3.429 * edge[0][1] * mm)
        end = (3.429 * edge[1][0] * mm, 3.429 * edge[1][1] * mm)
        dwg.add(dwg.line(start, end, stroke=svgwrite.rgb(255, 0, 0)))
    dwg.save()
    
def draw3dsvg(tree):
    layer1 = svgwrite.Drawing('layer1.svg', profile='full')
    layer2 = svgwrite.Drawing('layer2.svg', profile='full')
    layer3 = svgwrite.Drawing('layer3.svg', profile='full')
    mm = svgwrite.mm # 0.135 mm
    
    for edge in tree:
        start = (3.429 * edge[0][0] * mm, 3.429 * edge[0][1] * mm)
        end = (3.429 * edge[1][0] * mm, 3.429 * edge[1][1] * mm)
        if edge[0][2] == 0 and edge[1][2] == 0:
            layer1.add(layer1.line(start, end, stroke=svgwrite.rgb(255, 0, 0)))
        elif edge[0][2] == 1 and edge[1][2] == 1:
            layer3.add(layer3.line(start, end, stroke=svgwrite.rgb(255, 0, 0)))
        else:
            layer2.add(layer2.line(start, end, stroke=svgwrite.rgb(255, 0, 0)))
    layer1.save()
    layer2.save()
    layer3.save()

    

def random_solution(nodes, edges, start=(0,0), end=(N,N)):
    current = start
    path = []
    visited = set([None, current])
    while(current != end):
        found = False
        while found == False:
            connections = list(filter(lambda x: x[0] == current or x[1] == current, edges))
#             print(connections)
            step = random.sample(connections, 1)[0]
#             print(current, step, visited)
            print(current)
            
            if step[0] == current and step[1] not in visited:
                current = step[1]
                visited.add(current)
                found = True
            elif step[1] == current and step[0] not in visited:
                current = step[0]
                visited.add(current)
                found = True
    return path

In [34]:
mst = random_tree(nodes, edges)
drawsvg(mst, "40circle.svg")
# print(draw(mst, N, M)[0])

In [ ]:
f = open("6-mazes-final.txt", "w+")

for i in range(4**11):
    mst = random_tree(nodes, edges)
    pretty, numbers = draw(mst, N)
    if (i%1000 == 0):
        print(i)
    f.write(str(numbers) + "\n")

f.close()

In [ ]:
def get_pattern(teeth, cams, missings, debug=False):
    #which teeth are missing every wheel 
    #(i.e. if 5 is missing then the next wheel doesn't change when the prior wheel is on 5)
    prev = [0] * cams
    cur = [0] * cams
    count = 0;
    no_changes = 0;

    while True:
        count += 1
        cur[0] = (cur[0] + 1) % teeth
        for i in range(1, cams):
            if prev[i-1] != missings[i-1] and prev[i-1] != cur[i-1]:
                cur[i] = (cur[i] + 1)%teeth
            else:
                no_changes += 1
        s = "".join([str(i) for i in cur])
        for i in range(cams):
            prev[i] = cur[i]

        if debug:
            print(s, count)
        if s == "0"*cams:
            break
    return (count, no_changes, no_changes/(count * cams))


vals = dict()

def all_missings(teeth, cams, missings, idx):
    if idx == cams - 1:
        ans = get_pattern(teeth, cams, missings)
        if ans[0] not in vals:
            vals[ans[0]] = set([ans[1]])
            print(vals)
        elif ans[1] not in vals[ans[0]]:
            vals[ans[0]].add(ans[1])
            print(vals)
        return
    else:
        for i in range(teeth + 1):
            missings[idx] = i - 1
            all_missings(teeth, cams, missings, idx + 1)
        return

    
teeth = 3
cams = 7
# missings = [-1] * (cams-1)
get_pattern(teeth, cams, [2,0,1,2,0,1])
# all_missings(teeth, cams, missings, 0)

In [ ]:
# Maze Transition: transition matrix for which edges you can select on a given row without making a cycle

N = 22

def adj(nodea, nodeb):
    return (abs(nodea[0] - nodeb[0]) + abs(nodea[1] - nodeb[1]) == 1)

nodes = [(i,j) for i in range(N) for j in range(N)]
edges = [(nodes[i], nodes[j]) for i in range(N**2) for j in range(i, N**2) if adj(nodes[i], nodes[j])]
layers = []

for edge in edges:
    layers.append(edge[0][0] + edge[1][0])

print(edges)
print(layers)

for i in range(max(layers) + 1):
    for j in range(len(layers)):
        if layers[j] == i:
            print(edges[j])
    print()